In [1]:
from slim_gsgp_lib_np.main_slim import slim
from slim_gsgp_lib_np.utils.utils import train_test_split
from slim_gsgp_lib_np.utils.callbacks import *
from slim_gsgp_lib_np.evaluators.fitness_functions import rmse
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import time
import os
from tqdm import tqdm
from functions.test_funcs import mape, nrmse, r_squared, mae, standardized_rmse
from matplotlib import pyplot as plt
from slim_gsgp_lib_np.algorithms.SLIM_GSGP.operators.mutators import *
from slim_gsgp_lib_np.algorithms.SLIM_GSGP.operators.simplifiers import *
from slim_gsgp_lib_np.datasets.data_loader import *
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import pdist, squareform

datasets = [globals()[i] for i in globals() if 'load' in i][2:]

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["BLIS_NUM_THREADS"] = "1"


# -------------------------- # 
from slim_gsgp_lib_np.utils.utils import check_slim_version

In [34]:
# X, y = datasets[0]()
# X_train, X_test, y_train, y_test = train_test_split(X, y, p_test=0.2)
# X_train = MinMaxScaler().fit_transform(X_train)
# X_test = MinMaxScaler().fit_transform(X_test)
# y_train = MinMaxScaler().fit_transform(y_train.reshape(-1, 1)).ravel()
# y_test = MinMaxScaler().fit_transform(y_test.reshape(-1, 1)).ravel()

X = np.random.rand(20, 2)
y = np.random.rand(20)

X_train, X_test, y_train, y_test = train_test_split(X, y, p_test=0.2)

seed = 2

# agelog = LogAge()
# divlog = LogDiversity()
# early_stop = EarlyStopping_train(patience=2500)

example_tree, population = slim(X_train=X_train, y_train=y_train,
                    dataset_name='test', test_elite=False, slim_version='SLIM*ABS', # initializer='simple',
                    max_depth=14, init_depth=6, pop_size=100, n_iter=500, seed=seed, verbose=1,
                    p_inflate=0.3, p_struct=0.2, eps_fraction=1e-5,
                    prob_const=0.1, n_elites=1, selector='e_lexicase', 
                    decay_rate=0.2, p_xo=0, p_struct_xo=0, prob_terminal=0.8,
                    # callbacks=[agelog, divlog, early_stop], 
                    mode='exp', tournament_size=2, full_return=True, timeout=200,
    )

preds = example_tree.predict(X_test)
print('RMSE:', rmse(preds, y_test))

+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|     dataset     |        it       |      train      |       test      |       time      |      nodes      |       div       |     avgStru     |      avgDep     |      struct     |     inflate     |     deflate     |        xo       |
+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|       test      |        0        |      0.395      |       None      |      0.016      |        5        |        24       |      3.150      |      3.150      |     N/A (0)     |     N/A (0)     |     N/A (0)     |     N/A (0)     |
|-----------------|-----------------|-----------------|-

In [167]:
for ind in population.population:
    ind.version = example_tree.version
    ind.calculate_semantics(X_train)
    if '*' in ind.version:
        ind.calculate_errors_case(y_train, operator='mul')
    else:
        ind.calculate_errors_case(y_train, operator='sum')
    
    ind.train_semantics = ind.predict(X_train)
    ind.test_semantics = ind.predict(X_test)

In [194]:
def create_grow_random_tree(depth, 
                            FUNCTIONS, 
                            TERMINALS, 
                            CONSTANTS, 
                            p_c=0.3,
                            p_t=0.5, 
                            first_call=True):
    """
    Generates a random tree representation using the Grow method with a maximum specified depth.

    Parameters
    ----------
    depth : int
        Maximum depth of the tree to be created.
    FUNCTIONS : dict
        Dictionary of functions allowed in the tree.
    TERMINALS : dict
        Dictionary of terminal symbols allowed in the tree.
    CONSTANTS : dict
        Dictionary of constant values allowed in the tree.
    TERMINALS_KEYS : list
        Precomputed list of terminal keys.
    CONSTANTS_KEYS : list
        Precomputed list of constant keys.
    FUNCTIONS_KEYS : list
        Precomputed list of function keys.
    p_c : float, optional
        Probability of choosing a constant node. Default is 0.3.
    p_t : float, optional
        Probability of choosing a terminal node. Default is 0.5.
    first_call : bool, optional
        Variable that controls whether the function is being called for the first time. Default is True.

    Returns
    -------
    tuple or str
        The generated tree representation according to the specified parameters.
    """
    
    # If depth is 1 or a terminal is selected and it's not the first call
    if (depth <= 1 or random.random() < p_t) and not first_call:
        if random.random() > p_c:
            return random.choice(list(TERMINALS.keys()))
        else:
            return random.choice(list(CONSTANTS.keys()))
    
    # If a function is selected
    else:
        node = random.choice(list(FUNCTIONS.keys()))
        if FUNCTIONS[node]["arity"] == 2:
            left_subtree = create_grow_random_tree(depth - 1, FUNCTIONS, TERMINALS, CONSTANTS,
                                                   p_c, p_t, False)
            right_subtree = create_grow_random_tree(depth - 1, FUNCTIONS, TERMINALS, CONSTANTS,
                                                    p_c, p_t, False)
            return (node, left_subtree, right_subtree)
        else:
            left_subtree = create_grow_random_tree(depth - 1, FUNCTIONS, TERMINALS, CONSTANTS,
                                                   p_c, p_t, False)
            return (node, left_subtree)


def create_random_tree(depth_condition, depth_tree, FUNCTIONS, TERMINALS, CONSTANTS, SPECIALISTS,
                       p_specialist=0.3, p_t=0.5, p_c=0.3):
    """
    Generates a random ensemble tree representing an individual in the GP ensemble.
    
    The tree is structured as a conditional operator:
        (condition_tree, true_branch, false_branch)
    
    - condition_tree: generated using your existing create_grow_random_tree function.
    - true_branch / false_branch: either a further nested conditional tree or, with some probability,
      a specialist selected from SPECIALISTS.
    
    Parameters
    ----------
    depth : int
        Maximum depth for the ensemble tree.
    FUNCTIONS : dict
        Dictionary of function nodes (for condition trees).
    TERMINALS : dict
        Dictionary of terminal nodes (for condition trees).
    CONSTANTS : dict
        Dictionary of constant nodes (for condition trees).
    SPECIALISTS : dict
        Dictionary (or list) of specialist solutions.
    p_specialist : float, optional
        Probability of terminating a branch with a specialist rather than another conditional node.
    p_t : float, optional
        Terminal probability passed to create_grow_random_tree.
    p_c : float, optional
        Constant probability passed to create_grow_random_tree.
    
    Returns
    -------
    tuple or str
        A conditional tree or a specialist (when the branch is terminated).
    """
    # Base case: if depth is 0 or by chance we decide to return a specialist
    if depth_tree <= 0 or random.random() < p_specialist:
        return random.choice(list(SPECIALISTS.keys()))
    
    # Generate a condition tree.
    # Here we choose a random depth for the condition tree (at most the current depth)
    condition_tree = create_grow_random_tree(depth_condition, FUNCTIONS, TERMINALS, CONSTANTS,
                                             p_c=p_c, p_t=p_t, first_call=True)
    
    # Recursively build the true and false branches.
    true_branch = create_random_tree(depth_condition, depth_tree - 1, FUNCTIONS, TERMINALS, CONSTANTS, SPECIALISTS,
                                     p_specialist=p_specialist, p_t=p_t, p_c=p_c)
    false_branch = create_random_tree(depth_condition, depth_tree - 1, FUNCTIONS, TERMINALS, CONSTANTS, SPECIALISTS,
                                      p_specialist=p_specialist, p_t=p_t, p_c=p_c)
    
    return (condition_tree, true_branch, false_branch)

In [195]:
def tree_depth_and_nodes(FUNCTIONS, SPECIALISTS):
    """
    Returns a function that calculates three measures for a tree:
      1. Depth: length of the longest path from the root to a leaf.
      2. Node count: the total number of nodes in the tree (for function nodes, each is counted;
         ensemble nodes themselves are not counted as extra nodes).
      3. Ensemble nodes total: the sum of the node counts from the specialist trees used as terminals.
         For a specialist terminal, its individual node count is used (accessed via SPECIALISTS[key].nodes_count);
         non-specialist terminals contribute 0.

    The tree may be:
      - A function node (created by your grow method), represented as a tuple whose first element is a key in FUNCTIONS.
      - An ensemble (conditional) node, represented as a 3-tuple (condition, true_branch, false_branch)
        where the condition is a function tree and the branches are either ensemble nodes or terminals.
      - A terminal (string). If the string is a key in SPECIALISTS, its ensemble total is taken from the individual.
    
    Parameters
    ----------
    FUNCTIONS : dict
        Dictionary of function nodes allowed in the tree. Each function has an "arity" entry.
    SPECIALISTS : dict
        Dictionary of specialist individuals. For a specialist terminal key (e.g. "S_0"), its ensemble
        node count is retrieved by SPECIALISTS[key].nodes_count.
    
    Returns
    -------
    Callable
        A function that, given a tree, returns a triple (depth, node_count, ensemble_nodes_total).
    """
    def depth_and_nodes(tree, count_ensemble):
        # Base case: terminal node.
        if not isinstance(tree, tuple):
            if tree in SPECIALISTS:
                # Depth and node count are 1 for the terminal,
                # but the ensemble total comes from the specialist's own nodes_count.
                return 1, 1, SPECIALISTS[tree].nodes_count
            else:
                # Regular terminal (variable or constant): count as 1 node and depth 1, but no ensemble total.
                return 1, 1, 0

        # If the node is a function node (its first element is a key in FUNCTIONS)
        if isinstance(tree[0], str) and tree[0] in FUNCTIONS:
            arity = FUNCTIONS[tree[0]]["arity"]
            if arity == 2:
                d_left, n_left, ens_left = depth_and_nodes(tree[1], True)
                d_right, n_right, ens_right = depth_and_nodes(tree[2], True)
                depth_val = 1 + max(d_left, d_right)
                nodes_val = 1 + n_left + n_right
                ens_total = ens_left + ens_right
                return depth_val, nodes_val, ens_total
            elif arity == 1:
                d_child, n_child, ens_child = depth_and_nodes(tree[1], True)
                return 1 + d_child, 1 + n_child, ens_child
            else:
                # If the function node is of arity 0, treat it as a leaf.
                return 1, 1, 0

        else:
            # Otherwise, assume it's an ensemble (conditional) node: (condition, true_branch, false_branch)
            if len(tree) != 3:
                raise ValueError("Invalid ensemble tree structure. Expected a tuple of length 3.")
            d_cond, n_cond, ens_cond = depth_and_nodes(tree[0], False)
            d_true, n_true, ens_true = depth_and_nodes(tree[1], False)
            d_false, n_false, ens_false = depth_and_nodes(tree[2], False)
            # For depth, take the max depth among children; if this ensemble operator is counted, add 1.
            d = max(d_cond, d_true, d_false)
            if count_ensemble:
                d += 1
            # For node count, simply sum the node counts of the children (ensemble operator not counted as an extra node).
            n = n_cond + n_true + n_false
            # For ensemble total, sum the ensemble totals of the children.
            ens_total = ens_cond + ens_true + ens_false
            return d, n, ens_total

    return lambda tree: depth_and_nodes(tree, True)

In [208]:
def bound_value(vector, min_val, max_val):
    return np.clip(vector, min_val, max_val)

def _execute_gp_tree(repr_, X, FUNCTIONS, TERMINALS, CONSTANTS):
    if isinstance(repr_, tuple):  # If it's a function node
        function_name = repr_[0]
        if FUNCTIONS[function_name]["arity"] == 2:
            left_subtree, right_subtree = repr_[1], repr_[2]
            left_result = _execute_gp_tree(left_subtree, X, FUNCTIONS, TERMINALS,
                                        CONSTANTS)  # equivalent to Tree(left_subtree).apply_tree(inputs) if no parallelization were used
            right_result = _execute_gp_tree(right_subtree, X, FUNCTIONS, TERMINALS,
                                         CONSTANTS)  # equivalent to Tree(right_subtree).apply_tree(inputs) if no parallelization were used
            output = FUNCTIONS[function_name]["function"](
                left_result, right_result
            )
        else:
            left_subtree = repr_[1]
            left_result = _execute_gp_tree(left_subtree, X, FUNCTIONS, TERMINALS,
                                        CONSTANTS)  # equivalent to Tree(left_subtree).apply_tree(inputs) if no parallelization were used
            output = FUNCTIONS[function_name]["function"](left_result)

        return bound_value(output, -1e12, 1e12)

    else:  # If it's a terminal node
        if repr_ in TERMINALS:
            return X[:, TERMINALS[repr_]]
        elif repr_ in CONSTANTS:
            return np.full((X.shape[0],), CONSTANTS[repr_](None))

def _execute_tree(repr_, X, FUNCTIONS, TERMINALS, CONSTANTS, SPECIALISTS, testing=False, predict=False):
    """
    Evaluates a tree representation that may include ensemble (conditional) nodes.
    
    Ensemble nodes are represented as a 3-tuple:
         (condition, branch_if_true, branch_if_false)
    For the condition part, we use _execute_gp_tree to compute its semantics and then 
    create a mask (condition > 0). The mask is used to select, for each sample in X, which 
    branch to follow.
    
    Parameters
    ----------
    repr_ : tuple or str
        The tree representation. This may be:
          - A function node (tuple with the first element a key in FUNCTIONS),
          - An ensemble node (tuple with first element NOT in FUNCTIONS), or
          - A terminal (string).
    X : np.ndarray
        Input data samples (rows correspond to samples).
    FUNCTIONS : dict
        Dictionary of allowed functions for GP trees (each with an "arity" and "function").
    TERMINALS : dict
        Dictionary mapping terminal symbols to their corresponding column indices in X.
    CONSTANTS : dict
        Dictionary mapping constant symbols to constant-producing functions.
    SPECIALISTS : dict
        Dictionary mapping specialist keys to specialist individuals. Each specialist must have:
           - The precomputed attributes `train_semantics` and `test_semnatics`, and
           - A method `predict(X)` to compute semantics on new data.
    testing : bool, optional
        If True, use the specialist's precomputed test semantics. Else, use the train semantics.
    predict : bool, optional
        If True, use the specialist's predict method to compute semantics on new data.
    
    Returns
    -------
    np.ndarray
        A NumPy array of semantics for each sample in X.
    """
    # Check for ensemble (conditional) node:
    # We assume an ensemble node is a tuple whose first element is not a key in FUNCTIONS.
    if isinstance(repr_, tuple) and not (isinstance(repr_[0], str) and repr_[0] in FUNCTIONS):
        # Evaluate the condition using _execute_gp_tree.
        condition_semantics = _execute_gp_tree(repr_[0], X, FUNCTIONS, TERMINALS, CONSTANTS)
        # Create a Boolean mask: for each sample, True if condition > 0.
        mask = condition_semantics > 0
        
        # Evaluate the true and false branches recursively.
        true_branch = _execute_tree(repr_[1], X, FUNCTIONS, TERMINALS, CONSTANTS, SPECIALISTS, testing)
        false_branch = _execute_tree(repr_[2], X, FUNCTIONS, TERMINALS, CONSTANTS, SPECIALISTS, testing)
        # Combine the results: for each sample, pick the branch based on the mask.
        return np.where(mask, true_branch, false_branch)
    
    # Otherwise, if it is a standard function node, use _execute_gp_tree.
    if isinstance(repr_, tuple) and (isinstance(repr_[0], str) and repr_[0] in FUNCTIONS):
        return _execute_gp_tree(repr_, X, FUNCTIONS, TERMINALS, CONSTANTS)
    
    # Terminal node: check terminals, constants, and specialists.
    if not isinstance(repr_, tuple):
        if repr_ in TERMINALS:
            return X[:, TERMINALS[repr_]]
        elif repr_ in CONSTANTS:
            return np.full((X.shape[0],), CONSTANTS[repr_](None))
        elif repr_ in SPECIALISTS:
            if testing:
                return SPECIALISTS[repr_].test_semantics
            elif not predict:
                return SPECIALISTS[repr_].train_semantics
            else: 
                return SPECIALISTS[repr_].predict(X)
        else:
            raise ValueError("Unknown terminal symbol: " + str(repr_))

In [209]:
class Tree:
    """
    The Tree class representing the candidate solutions in MULTI-SLIM-GSGP.

    Attributes
    ----------
    repr_ : tuple or str
        Representation of the tree structure.
    FUNCTIONS : dict
        Dictionary of allowed functions in the tree representation.
    TERMINALS : dict
        Dictionary of terminal symbols allowed in the tree representation.
    CONSTANTS : dict
        Dictionary of constant values allowed in the tree representation.
    depth : int
        Depth of the tree.
    fitness : float
        Fitness value of the tree.
    test_fitness : float
        Test fitness value of the tree.
    node_count : int
        Number of nodes in the tree.
    """

    TERMINALS = None
    FUNCTIONS = None
    CONSTANTS = None
    SPECIALISTS = None

    def __init__(self, repr_):
        """
        Initializes a Tree object.

        Parameters
        ----------
        repr_ : tuple
            Representation of the tree structure.
        """
        self.FUNCTIONS = Tree.FUNCTIONS
        self.TERMINALS = Tree.TERMINALS
        self.CONSTANTS = Tree.CONSTANTS
        self.SPECIALISTS = Tree.SPECIALISTS

        self.repr_ = repr_
        self.depth, self.nodes_count, self.total_nodes = tree_depth_and_nodes(Tree.FUNCTIONS, Tree.SPECIALISTS)(repr_)
        self.fitness = None
        self.test_fitness = None

    def apply_tree(self, inputs, testing, predict):
        return _execute_tree(
            repr_=self.repr_,
            X=inputs,
            FUNCTIONS=self.FUNCTIONS,
            TERMINALS=self.TERMINALS,
            CONSTANTS=self.CONSTANTS,
            SPECIALISTS=self.SPECIALISTS,
            testing=testing,
            predict=predict
        )

    def evaluate(self, ffunction, X, y, testing=False):
        preds = self.apply_tree(X, testing, False)

        if testing:
            self.test_fitness = ffunction(y, preds)
        else:
            self.fitness = ffunction(y, preds)        

    def predict(self, X):
        return self.apply_tree(X, testing=False, predict=True)

    def get_tree_representation(self, indent=""):
        representation = []
        
        if isinstance(self.repr_, tuple):
            # Check if the node is a function node (its first element is a function key)
            if isinstance(self.repr_[0], str) and self.repr_[0] in Tree.FUNCTIONS:
                function_name = self.repr_[0]
                representation.append(indent + f"{function_name}(\n")
                
                # Process children based on the function's arity.
                if Tree.FUNCTIONS[function_name]["arity"] == 2:
                    left_subtree, right_subtree = self.repr_[1], self.repr_[2]
                    representation.append(Tree(left_subtree).get_tree_representation(indent + "  "))
                    representation.append(Tree(right_subtree).get_tree_representation(indent + "  "))
                else:
                    left_subtree = self.repr_[1]
                    representation.append(Tree(left_subtree).get_tree_representation(indent + "  "))
                
                representation.append(indent + ")\n")
            else:
                # Otherwise, assume it is a conditional node.
                # Structure: (condition, branch_if_true, branch_if_false)
                representation.append(indent + "if (\n")
                representation.append(Tree(self.repr_[0]).get_tree_representation(indent + "  "))
                representation.append(indent + ") > 0 then\n")
                representation.append(Tree(self.repr_[1]).get_tree_representation(indent + "  "))
                representation.append(indent + "else\n")
                representation.append(Tree(self.repr_[2]).get_tree_representation(indent + "  "))
                representation.append(indent + "endif\n")
        else:
            # Terminal node.
            representation.append(indent + f"{self.repr_}\n")
        
        return "".join(representation)

    def print_tree_representation(self, indent=""):
        print(self.get_tree_representation(indent=indent))


    def get_forest_representation(self):
        """
        Returns a LaTeX string using the forest package to display the tree.
        
        Function nodes are printed as:
        [function_name child1 child2]
        
        Conditional nodes (assumed to have the structure (condition, branch_if_true, branch_if_false))
        are printed as:
        [{\textbf{if}}
            [{Condition:} condition]
            [{\textbf{then}} true_branch]
            [{\textbf{else}} false_branch]
        ]
        
        Terminals are printed as:
        [\textit{terminal}]
        
        Returns
        -------
        str
            A string containing the complete forest environment with the tree.
        """
        def rec(tree):
            if isinstance(tree, tuple):
                # Check if it's a function node (first element in Tree.FUNCTIONS)
                if isinstance(tree[0], str) and tree[0] in Tree.FUNCTIONS:
                    func_name = tree[0]
                    arity = Tree.FUNCTIONS[func_name]["arity"]
                    if arity == 2:
                        return f"[{func_name} {rec(tree[1])} {rec(tree[2])}]"
                    else:
                        return f"[{func_name} {rec(tree[1])}]"
                else:
                    # Otherwise, assume it's a conditional (ensemble) node.
                    return ("[{\\textbf{if}} " +
                            "[{Condition:} " + rec(tree[0]) + "] " +
                            "[{\\textbf{then}} " + rec(tree[1]) + "] " +
                            "[{\\textbf{else}} " + rec(tree[2]) + "]" +
                            "]")
            else:
                # Terminal: wrap the value in \textit{}
                return f"[\\textit{{{tree}}}]"
        
        forest_tree = rec(self.repr_)
        return ("\\begin{forest}\n"
                "  for tree={fit=band},\n"
                f"{forest_tree}\n"
                "\\end{forest}")

    def print_forest_representation(self):
        """
        Prints the LaTeX forest representation of the tree.
        """
        print(self.get_forest_representation())


In [210]:
FUNCTIONS = example_tree.collection[0].FUNCTIONS
TERMINALS = example_tree.collection[0].TERMINALS
CONSTANTS = example_tree.collection[0].CONSTANTS
SPECIALISTS = {f'S_{i}' : ind for i, ind in enumerate(population.population) if i<5}
Tree.FUNCTIONS = FUNCTIONS
Tree.TERMINALS = TERMINALS
Tree.CONSTANTS = CONSTANTS
Tree.SPECIALISTS = SPECIALISTS

In [211]:
struc = create_random_tree(2, 1, FUNCTIONS, TERMINALS, CONSTANTS, SPECIALISTS)
print(struc)
a = _execute_tree(struc, X_test, FUNCTIONS, TERMINALS, CONSTANTS, SPECIALISTS, testing=True)
b = _execute_tree(struc, X_train, FUNCTIONS, TERMINALS, CONSTANTS, SPECIALISTS, testing=False)
print('RMSE (test) before:', rmse(preds, y_test))
print('RMSE (test) ensemble:', rmse(a, y_test))
print('\n')
print('RMSE (train) before:', rmse(example_tree.predict(X_train), y_train))
print('RMSE (train) ensemble:', rmse(b, y_train))

(('divide', 'x0', 'x0'), 'S_0', 'S_1')
RMSE (test) before: 0.2676382694172165
RMSE (test) ensemble: 0.2676382694172165


RMSE (train) before: 0.29893853821542554
RMSE (train) ensemble: 0.29893853821542554


In [174]:
rmse(errors=SPECIALISTS['S_1'].errors_case_test)

np.float64(0.4893720639988294)

In [191]:
_execute_tree(struc, X_train, FUNCTIONS, TERMINALS, CONSTANTS, SPECIALISTS, train_semantics=True)

array([0.75571417, 0.75571417, 0.75571417, 0.75571417, 0.75571417,
       0.75571417, 0.75571417, 0.75571417, 0.75571417, 0.75571417,
       0.75571417, 0.75571417, 0.75571417, 0.75571417, 0.75571417,
       0.75571417])

In [193]:
SPECIALISTS['S_1'].train_semantics

array([0.75571417, 0.75571417, 0.75571417, 0.75571417, 0.75571417,
       0.75571417, 0.75571417, 0.75571417, 0.75571417, 0.75571417,
       0.75571417, 0.75571417, 0.75571417, 0.75571417, 0.75571417,
       0.75571417])